# Additional Insert Statements

Although we have covered the INSERT statement, we need to consider some of the quirks that come along with having a different construct of table to enter data into.

For example, how do we deal with INSERTs when we have Identity Columns, Default Values, and Calculated Columns?  These all change how we need to think about our INSERTS.

#### Identity Columns

During the Table Creation section we specified that we wished to see a MemberNumber starting at 1 million and incrementing in 5s.

To achieve this we defined an Identity within our Table Creation statement:

In [ ]:
drop table if exists #memberTable;

create table #memberTable
(
    memberNumber int identity(1000000, 5),
    firstName varchar(25),
    lastName varchar(35),
    fullName varchar(60),
    isActive bit
);

So how do we INSERT into this table?

In our last example for INSERT we were utilising a ROW\_NUMBER to generate our own Identity, but we know that this has to start at 1 and increment by 1. Therefore it is unsuitable. And, if we have told SQL Server that the memberNumber column looks after its own Identity values, will we even be able to override this?

Let's try. We can use our last statement from the previous demo to try and INSERT into this table using our ROW\_NUMBER query:

In [ ]:
drop table if exists #memberTable;

create table #memberTable
(
    memberNumber int identity(1000000, 5),
    firstName varchar(25),
    lastName varchar(35),
    fullName varchar(60),
    isActive bit
);

insert into #memberTable
(
    memberNumber, firstName, lastName, fullName, isActive
)
select row_number() over(order by firstName) as memberNumber,
        firstName, lastName, CONCAT_WS(' ', firstName, lastName) as fullName,
        1 as isActive
from JupyterDatabase.hr.employee
where employeeID <= 5;

As you can see, this fails.  We receive an error stating:

<code>Cannot insert explicit value for identity column in table '#memberTable' when IDENTITY_INSERT is set to OFF.</code>

This tells us that we cannot INSERT into the Identity column.  (It tells us there is a setting that does allow, but this is not something we will be covering as it should not be necessary at the basic T-SQL level).

So let's remove the ROW_NUMBER entirely and see if this allows us to INSERT.  Also note that, as we are reducing the SELECT list, we will also remove the column memberNumber from the INSERT list too:

In [ ]:
drop table if exists #memberTable;

create table #memberTable
(
    memberNumber int identity(1000000, 5),
    firstName varchar(25),
    lastName varchar(35),
    fullName varchar(60),
    isActive bit
);

insert into #memberTable
(
    firstName, lastName, fullName, isActive
)
select firstName, lastName, CONCAT_WS(' ', firstName, lastName) as fullName,
        1 as isActive
from JupyterDatabase.hr.employee
where employeeID <= 5;

select *
from #memberTable;

This works perfectly and we can clearly see our Identity column working exactly as defined.

Therefore we need to remember that if we have an Identity column in our table then we cannot write directly into that column and need to ensure that it is also removed from the INSERT list of columns.

#### Default Values

Now that we have dealt with our Identity column we shall revisit our DEFAULT.

In these examples we will utilise the simpler syntax for adding a default in order to make our code more concise and readable for the sake of learning:

In [ ]:
drop table if exists #memberTable;

create table #memberTable
(
    memberNumber int identity(1000000, 5),
    firstName varchar(25),
    lastName varchar(35),
    fullName varchar(60),
    isActive bit default(1)
);

As we discussed in the Table Creation section, a default means that if we do not enter a value into the column then SQL Server will enter a value for us.

Analysing this comment tells us that we CAN insert into a default, we simply don't HAVE to.

Therefore if we run the query above (inserting all data with the exception only of the Identity) then, considering the above, it feels as though this should work:

In [ ]:
drop table if exists #memberTable;

create table #memberTable
(
    memberNumber int identity(1000000, 5),
    firstName varchar(25),
    lastName varchar(35),
    fullName varchar(60),
    isActive bit default(1)
);

insert into #memberTable
(
    firstName, lastName, fullName, isActive
)
select firstName, lastName, CONCAT_WS(' ', firstName, lastName) as fullName,
        1 as isActive
from JupyterDatabase.hr.employee
where employeeID <= 5;

select *
from #memberTable;

And it does indeed work as expected.

So why the Default?  Well, it means that we don't HAVE to pass a value to SQL Server for the column in question.

We already know that the number of columns in our SELECT line must match the number of columns in our INSERT line, so let's remove 'isActive' from both and see what happens:

In [ ]:
drop table if exists #memberTable;

create table #memberTable
(
    memberNumber int identity(1000000, 5),
    firstName varchar(25),
    lastName varchar(35),
    fullName varchar(60),
    isActive bit default(1)
);

insert into #memberTable
(
    firstName, lastName, fullName
)
select firstName, lastName, CONCAT_WS(' ', firstName, lastName) as fullName
from JupyterDatabase.hr.employee
where employeeID <= 5;

select *
from #memberTable;

As before, this works perfectly well.

Therefore we can see that a Default Value can be very useful as, unlike an Identity, we do have the ability to override it if we wish, but if we choose not to enter anything into that column (and remove it from our INSERT list as well as our SELECT list) then SQL Server will fill in the data for us.

#### Calculated Columns

The last type of column we met within Table Creation was the Calculated Column.

If you remember, we used the table itself to define how a column should look based on other columns (in this case determining fullName based on firstName and lastName):

In [ ]:
drop table if exists #memberTable;

create table #memberTable
(
    memberNumber int identity(1000000, 5),
    firstName varchar(25),
    lastName varchar(35),
    fullName as concat_ws(' ', firstName, lastName),
    isActive bit default(1)
);

The first thing we can note here is that we have no datatype on the column which makes it feel like we cannot place anything into it.  However, that's never stopped us before so let's run our previous statement and see what happens:

In [ ]:
drop table if exists #memberTable;

create table #memberTable
(
    memberNumber int identity(1000000, 5),
    firstName varchar(25),
    lastName varchar(35),
    fullName as concat_ws(' ', firstName, lastName),
    isActive bit default(1)
);

insert into #memberTable
(
    firstName, lastName, fullName
)
select firstName, lastName, CONCAT_WS(' ', firstName, lastName) as fullName
from JupyterDatabase.hr.employee
where employeeID <= 5;

select *
from #memberTable;

So, we get an error telling us that:

<code>The column "fullName" cannot be modified because it is either a computed column or is the result of a UNION operator.</code>

Therefore we cannot modify this column.  What this means is that we cannot insert into this column, edit any perceived data (as the data in this column doesn't actually exist in SQL Server.  The database engine actually creates the dat on the fly as it returns it to the user)

This leaves us with our usual response in which we remove the column from both our SELECT and our INSERT column lists as we now know that SQL Server will take care of it for us.

Let's try this:

In [ ]:
drop table if exists #memberTable;

create table #memberTable
(
    memberNumber int identity(1000000, 5),
    firstName varchar(25),
    lastName varchar(35),
    fullName as concat_ws(' ', firstName, lastName),
    isActive bit default(1)
);

insert into #memberTable
(
    firstName, lastName
)
select firstName, lastName
from JupyterDatabase.hr.employee
where employeeID <= 5;

select *
from #memberTable;

Once again, this method works.

We can also now see how powerful a good database design can be when working with data and with reference to applications and users which may interact with it.

As per our last demo, we have a table of 5 columns and yet can populate it by passing in just 2.  Not only is this more concise and cleaner, but it also removes the potential for user error (manual typing of identities or mis-spelling a member's full name), and supports an easier application interface as well.

This type of design can also prove useful when building reports in which we may only receive first and last names, for example, yet all our outputs wish to return fullName and we do not want to manually write that piece of code each time, allowing for potential errors to occur.